### Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### ML Client session

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

registry_name = 'azureml'
credential = DefaultAzureCredential()
ml_client =  MLClient.from_config(credential=credential)

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name
)

### Numeric and Categorical columns

In [ ]:
import pandas as pd

target_column = 'readmit_status'

def get_categorical_numerical_data(dataset):
    dataset = dataset.drop([target_column], axis = 1)  
    categorical = []
    for col, value in dataset.iteritems():
        if value.dtype == 'object' or value.dtype == 'bool':
            categorical.append(col)
    numerical = dataset.columns.difference(categorical)
    return categorical, numerical

# get categorical and numerical fields from training data
train_data = pd.read_parquet('data/training_data.parquet')
categorical, numerical = get_categorical_numerical_data(train_data)
print('categorical columns: ',  categorical)
print('numerical field: ', numerical)

### Dashboard components

In [ ]:
label = 'latest'

rai_constructor_component = ml_client_registry.components.get(
    name='microsoft_azureml_rai_tabular_insight_constructor', label=label
)

# We get latest version and use the same version for all components
version = rai_constructor_component.version

rai_explanation_component = ml_client_registry.components.get(
    name='microsoft_azureml_rai_tabular_explanation', version=version
)

rai_erroranalysis_component = ml_client_registry.components.get(
    name='microsoft_azureml_rai_tabular_erroranalysis', version=version
)

rai_gather_component = ml_client_registry.components.get(
    name='microsoft_azureml_rai_tabular_insight_gather', version=version
)